In [1]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=e947331445189f6dd402b8d245e722b24b8cf29632da8d3a65696f56e62c4b50
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [13]:
df = pd.read_csv('https://raw.githubusercontent.com/ddpasiliao/AI_First_Day_4/refs/heads/main/SupplyChainData.csv')

In [14]:
df.sample(5)

,Item_ID,Item_Name,Quantity,Weight_kg,Dimensions_cm (L x W x H),Fragile,Stackable,Delivery_Location,Vehicle_Type,Vehicle_Capacity_kg,Vehicle_Dimensions_cm (L x W x H)
11,12,Refrigerators,5,75.0,70 x 80 x 180,Yes,No,Cebu,Truck,"5,000",400 x 200 x 200
19,20,Carpets,30,8.0,200 x 20 x 20,No,Yes,Manila,Truck,"5,000",400 x 200 x 200
3,4,Monitors,40,3.0,60 x 30 x 20,Yes,Yes,Manila,Van,"1,000",300 x 150 x 150
8,9,Paper Rolls,200,1.0,10 x 10 x 20,No,Yes,Davao,Container,"20,000",800 x 240 x 240
10,11,Smart TVs,10,8.0,100 x 20 x 60,Yes,No,Manila,Van,"1,000",300 x 150 x 150


## Understanding the Dataset

In [15]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info,"\n")


Item_ID                                int64
Item_Name                             object
Quantity                               int64
Weight_kg                            float64
Dimensions_cm (L x W x H)             object
Fragile                               object
Stackable                             object
Delivery_Location                     object
Vehicle_Type                          object
Vehicle_Capacity_kg                   object
Vehicle_Dimensions_cm (L x W x H)     object
dtype: object 



In [16]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("Categorical Features:", categorical_features)
print("Numerical Features:", numerical_features)



Categorical Features: Index(['Item_Name', 'Dimensions_cm (L x W x H)', 'Fragile', 'Stackable',
       'Delivery_Location', 'Vehicle_Type', 'Vehicle_Capacity_kg',
       'Vehicle_Dimensions_cm (L x W x H)'],
      dtype='object')
Numerical Features: Index(['Item_ID', 'Quantity', 'Weight_kg'], dtype='object')


In [17]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    unique_values = df[col].unique().tolist() # get unique values and transform into list
    if len(unique_values) > max_display:
        return unique_values[:max_display] + ['...']
    return unique_values

unique_values_for_categorical = {col: get_unique_values(col) for col in categorical_features}
print(unique_values_for_categorical)

for col, unique_values in unique_values_for_categorical.items():
    print(f"Unique values for {col}:")
    for value in unique_values:
        print(f"- {value}")



{'Item_Name': ['Laptops', 'Office Chairs', 'Desks', 'Monitors', 'Books', 'Printers', 'Filing Cabinets', 'Air Conditioners', 'Paper Rolls', 'Desktop Computers', '...'], 'Dimensions_cm (L x W x H)': ['30 x 20 x 5', '80 x 80 x 100', '150 x 80 x 75', '60 x 30 x 20', '25 x 20 x 5', '50 x 50 x 30', '60 x 60 x 120', '70 x 60 x 35', '10 x 10 x 20', '40 x 20 x 40', '...'], 'Fragile': ['Yes', 'No'], 'Stackable': ['No', 'Yes'], 'Delivery_Location': ['Manila', 'Cebu', 'Davao'], 'Vehicle_Type': ['Truck', 'Container', 'Van'], 'Vehicle_Capacity_kg': ['5,000', '20,000', '1,000'], 'Vehicle_Dimensions_cm (L x W x H)': ['400 x 200 x 200', '800 x 240 x 240', '300 x 150 x 150']}
Unique values for Item_Name:
- Laptops
- Office Chairs
- Desks
- Monitors
- Books
- Printers
- Filing Cabinets
- Air Conditioners
- Paper Rolls
- Desktop Computers
- ...
Unique values for Dimensions_cm (L x W x H):
- 30 x 20 x 5
- 80 x 80 x 100
- 150 x 80 x 75
- 60 x 30 x 20
- 25 x 20 x 5
- 50 x 50 x 30
- 60 x 60 x 120
- 70 x 60 x 

In [18]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")



Descriptive Statistics for Numerical Features:
         Item_ID    Quantity  Weight_kg
count  20.00000   20.000000  20.000000
mean   10.50000   56.750000  14.865000
std     5.91608   57.314346  19.775085
min     1.00000    5.000000   0.500000
25%     5.75000   18.750000   1.875000
50%    10.50000   35.000000   8.000000
75%    15.25000   70.000000  16.250000
max    20.00000  200.000000  75.000000 



## Generating the Template

In [24]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f"{col}: {dtype}\n"


    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, unique_values in unique_values_for_categorical.items():
        unique_values_str += f"{col}: {unique_values_for_categorical}\n"



    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f"{col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"{stat_name}: {value}\n"



    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """


    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."


    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly
    """


    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}") # e is the error log
            time.sleep(2)  # Wait before retrying

    return None


In [35]:
from tempfile import template

# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)


"The item '{Item_Name}' with an ID of {Item_ID} has a quantity of {Quantity} and weighs {Weight_kg} kg. Its dimensions are {Dimensions_cm (L x W x H)}, and it is marked as {Fragile} and {Stackable}. This item is scheduled for delivery to {Delivery_Location} using a {Vehicle_Type} with a capacity of {Vehicle_Capacity_kg} and dimensions of {Vehicle_Dimensions_cm (L x W x H)}."


## Populating the Template with Actual Values

In [40]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph

In [42]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

# print the paragraph
print(df['paragraph'])

0     "The item 'Laptops' with an ID of 1 has a quan...
1     "The item 'Office Chairs' with an ID of 2 has ...
2     "The item 'Desks' with an ID of 3 has a quanti...
3     "The item 'Monitors' with an ID of 4 has a qua...
4     "The item 'Books' with an ID of 5 has a quanti...
5     "The item 'Printers' with an ID of 6 has a qua...
6     "The item 'Filing Cabinets' with an ID of 7 ha...
7     "The item 'Air Conditioners' with an ID of 8 h...
8     "The item 'Paper Rolls' with an ID of 9 has a ...
9     "The item 'Desktop Computers' with an ID of 10...
10    "The item 'Smart TVs' with an ID of 11 has a q...
11    "The item 'Refrigerators' with an ID of 12 has...
12    "The item 'Medical Supplies' with an ID of 13 ...
13    "The item 'Tires' with an ID of 14 has a quant...
14    "The item 'Solar Panels' with an ID of 15 has ...
15    "The item 'Plastic Crates' with an ID of 16 ha...
16    "The item 'Lighting Fixtures' with an ID of 17...
17    "The item 'Cement Bags' with an ID of 18 h

In [44]:
df['paragraph'][0]

'"The item \'Laptops\' with an ID of 1 has a quantity of 50 and weighs 1.5 kg. Its dimensions are 30 x 20 x 5, and it is marked as Yes and No. This item is scheduled for delivery to Manila using a Truck with a capacity of 5,000 and dimensions of 400 x 200 x 200."'

In [45]:
df.sample(5)

,Item_ID,Item_Name,Quantity,Weight_kg,Dimensions_cm (L x W x H),Fragile,Stackable,Delivery_Location,Vehicle_Type,Vehicle_Capacity_kg,Vehicle_Dimensions_cm (L x W x H),paragraph
1,2,Office Chairs,20,15.0,80 x 80 x 100,No,Yes,Cebu,Truck,"5,000",400 x 200 x 200,"""The item 'Office Chairs' with an ID of 2 has ..."
10,11,Smart TVs,10,8.0,100 x 20 x 60,Yes,No,Manila,Van,"1,000",300 x 150 x 150,"""The item 'Smart TVs' with an ID of 11 has a q..."
5,6,Printers,25,10.0,50 x 50 x 30,Yes,No,Davao,Container,"20,000",800 x 240 x 240,"""The item 'Printers' with an ID of 6 has a qua..."
19,20,Carpets,30,8.0,200 x 20 x 20,No,Yes,Manila,Truck,"5,000",400 x 200 x 200,"""The item 'Carpets' with an ID of 20 has a qua..."
9,10,Desktop Computers,30,5.0,40 x 20 x 40,Yes,No,Manila,Truck,"5,000",400 x 200 x 200,"""The item 'Desktop Computers' with an ID of 10..."


## Saving the Populated Templates (Natural Language Texts)

In [66]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [67]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")


In [70]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

